# Data Preprocessing Pipeline for Chest X-Ray Classification

This notebook performs all data preprocessing steps required for training the CheXNet models.
We will:
1. Load and filter the metadata CSV
2. Map images to their file paths
3. Process multi-label classifications
4. Create train/validation splits
5. Save preprocessed data for model training

## Step 1: Import Libraries
**Explanation:** We import all necessary libraries for data manipulation, file handling, and visualization.

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## Step 2: Define Paths and Constants
**Explanation:** We define the paths to our data files and the list of disease labels we want to classify.

In [ ]:
# Paths
BASE_DIR = '.'
CSV_PATH = os.path.join(BASE_DIR, 'Data_Entry_2017_v2020.csv')
IMAGE_DIRS = [
    os.path.join(BASE_DIR, 'images'),
    os.path.join(BASE_DIR, 'images 2')
]

# Disease labels (14 classes)
ALL_LABELS = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 
    'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 
    'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 
    'Pleural_Thickening', 'Hernia'
]

print(f"CSV Path: {CSV_PATH}")
print(f"Image Directories: {IMAGE_DIRS}")
print(f"Number of disease classes: {len(ALL_LABELS)}")

## Step 3: Load Metadata CSV
**Explanation:** We load the CSV file containing metadata about each X-ray image, including patient information and disease labels.

In [ ]:
# Load CSV
df = pd.read_csv(CSV_PATH)

print(f"Original dataset size: {len(df)} rows")
print(f"\nColumns in dataset:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

## Step 4: Map Images to File Paths
**Explanation:** We scan the image directories and create a mapping from image filenames to their full file paths. This allows us to quickly locate each image file.

In [ ]:
# Create image path mapping
image_paths = {}

for img_dir in IMAGE_DIRS:
    if os.path.exists(img_dir):
        print(f"Scanning directory: {img_dir}")
        for fname in os.listdir(img_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths[fname] = os.path.join(img_dir, fname)

print(f"\nTotal images found: {len(image_paths)}")
print(f"Sample mapping:")
print(list(image_paths.items())[:3])

## Step 5: Filter Dataset to Available Images
**Explanation:** We filter the metadata to only include rows where we have the corresponding image file. This ensures we don't try to train on missing data.

In [ ]:
# Map image paths to dataframe
df['path'] = df['Image Index'].map(image_paths)

# Count missing images
missing_count = df['path'].isna().sum()
print(f"Images without files: {missing_count}")

# Remove rows with missing images
df = df.dropna(subset=['path'])

print(f"Dataset size after filtering: {len(df)} rows")
print(f"Removed {missing_count} rows with missing images")

## Step 6: Clean Patient Age Data
**Explanation:** We convert the 'Patient Age' column to numeric and remove any outliers (e.g., ages > 100) to ensure data quality.

In [ ]:
# Convert age to numeric
df['Patient Age'] = pd.to_numeric(df['Patient Age'], errors='coerce')

# Check for invalid ages
invalid_ages = df[df['Patient Age'] > 100]
print(f"Rows with age > 100: {len(invalid_ages)}")

# Remove invalid ages
df = df[df['Patient Age'] < 100]

print(f"\nAge statistics after cleaning:")
print(df['Patient Age'].describe())

## Step 7: Process Multi-Label Classifications
**Explanation:** Each image can have multiple diseases (multi-label). We convert the pipe-separated string (e.g., "Infiltration|Effusion") into binary columns for each disease.

In [ ]:
# Create binary columns for each disease
for label in ALL_LABELS:
    df[label] = df['Finding Labels'].map(lambda x: 1.0 if label in x else 0.0)

print("Created binary label columns for each disease.")
print(f"\nLabel distribution:")
label_counts = df[ALL_LABELS].sum().sort_values(ascending=False)
print(label_counts)

# Visualize label distribution
plt.figure(figsize=(12, 6))
label_counts.plot(kind='bar', color='steelblue')
plt.title('Disease Label Distribution')
plt.ylabel('Count')
plt.xlabel('Disease')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Step 8: Analyze Class Imbalance
**Explanation:** We check how balanced our dataset is. Medical datasets are often highly imbalanced, which affects model training.

In [ ]:
# Calculate positive percentage for each class
class_balance = (df[ALL_LABELS].sum() / len(df) * 100).sort_values(ascending=False)

print("Percentage of positive samples per class:")
for label, pct in class_balance.items():
    print(f"{label:20s}: {pct:6.2f}%")

# Visualize imbalance
plt.figure(figsize=(12, 6))
class_balance.plot(kind='barh', color='coral')
plt.title('Class Imbalance (% of Positive Samples)')
plt.xlabel('Percentage (%)')
plt.tight_layout()
plt.show()

## Step 9: Create Train/Validation Split
**Explanation:** We split the data into training (80%) and validation (20%) sets. We use a fixed random seed to ensure reproducibility.

In [ ]:
# Perform train/val split
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42,
    stratify=None  # Can't stratify multi-label easily
)

print(f"Training set size: {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
print(f"Validation set size: {len(val_df)} ({len(val_df)/len(df)*100:.1f}%)")

# Verify split maintains label distribution
print("\nLabel distribution in training set:")
print(train_df[ALL_LABELS].sum())
print("\nLabel distribution in validation set:")
print(val_df[ALL_LABELS].sum())

## Step 10: Save Preprocessed Data
**Explanation:** We save the preprocessed train and validation dataframes to CSV files for easy loading during model training.

In [ ]:
# Save preprocessed data
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)

print("Saved preprocessed data:")
print(f"  - train_data.csv ({len(train_df)} rows)")
print(f"  - val_data.csv ({len(val_df)} rows)")

## Step 11: Summary Statistics
**Explanation:** Final summary of the preprocessed dataset.

In [ ]:
print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)
print(f"Total images processed: {len(df)}")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Number of disease classes: {len(ALL_LABELS)}")
print(f"\nPatient demographics:")
print(f"  Age range: {df['Patient Age'].min():.0f} - {df['Patient Age'].max():.0f}")
print(f"  Mean age: {df['Patient Age'].mean():.1f}")
print(f"  Gender distribution:")
print(df['Patient Sex'].value_counts())
print(f"\nView position distribution:")
print(df['View Position'].value_counts())
print("="*60)
print("Preprocessing complete! Data ready for model training.")
print("="*60)